In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# use a fake header
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}

In [3]:
start = "https://www.indeed.com/jobs?q=Data+Scientist&l=Denver%2C+CO"

# page nbr argument 1-10, 11-20, 

In [4]:
test = requests.get(start, headers=headers)

In [5]:
soup = BeautifulSoup(test.text, "html.parser")
links = soup.find_all("a")

In [20]:
# build a list of links
some_links = []

for l in links:
    try:
        hyperlink = l.attrs.get('href')
        if "/rc/clk?" in hyperlink:
            some_links.append(l.attrs.get('href'))
    except:
        pass

In [24]:
job_links = ["https://www.indeed.com{}".format(x)
             for x in some_links
             ]

In [25]:
job_links

['https://www.indeed.com/rc/clk?jk=da4412e0f36920a0&fccid=de71a49b535e21cb&vjs=3',
 'https://www.indeed.com/rc/clk?jk=82d5f1b9af90d82f&fccid=0a0a2172bfedd7b7&vjs=3',
 'https://www.indeed.com/rc/clk?jk=bdf4b60a20a918fd&fccid=3c371b712ace44cf&vjs=3',
 'https://www.indeed.com/rc/clk?jk=c9d84c941c6b6d2f&fccid=4600afcfb0f58ad6&vjs=3',
 'https://www.indeed.com/rc/clk?jk=1fce0c6ca709f5ef&fccid=3a13e586bee51cc7&vjs=3',
 'https://www.indeed.com/rc/clk?jk=128ca9d1ab4137b0&fccid=49bacaf9ea5daa22&vjs=3',
 'https://www.indeed.com/rc/clk?jk=b0449613d0b71523&fccid=7cd7119315ca0934&vjs=3',
 'https://www.indeed.com/rc/clk?jk=f3ec178c7a762e5a&fccid=d512f2c02718adce&vjs=3',
 'https://www.indeed.com/rc/clk?jk=3b7429025e95323f&fccid=3c74eafe288fc8ca&vjs=3']

In [26]:
test = requests.get(job_links[0], headers=headers)
soup = BeautifulSoup(test.text, "html.parser")


In [31]:
spans = soup.find_all('span')
spans_w_divs = [span.find_all('div') for span in spans if len(span.find_all('div')) > 0]

In [33]:
spans_w_divs

[[<div><b>Job Description</b><br/>
  IBM Global Business Services (GBS) is a team of business, strategy and technology consultants enabling enterprises to make smarter decisions and providing unparalleled client and consumer experiences in cognitive, data analytics, cloud technology and mobile app development. With global reach, outcome-focused methodologies and deep industry expertise, IBM GBS empowers clients to digitally reinvent their business and get the competitive edge in the cognitive era in over 170 countries.<br/>
  <br/>
  Bottom line? We outthink ordinary. Discover what you can do at IBM.<br/>
  <br/>
  <b>THIS POSITION REQUIRES AN ACTIVE TS/SCI CLEARANCE WITH A CI POLYGRAPH</b><br/>
  <br/>
  We are seeking an experienced data scientist with software development skills to join the team of IBMers currently supporting a government client. The government client is working to redefine the business of delivering intelligence data and analysis. IBM has worked closely with its cl

In [38]:
span = soup.find("span", id="job_summary")

In [44]:
from IPython.core.display import HTML
display(HTML(str(span)))

In [28]:
print(soup.get_text())



Analytic Data Scientist job - IBM - Denver, CO | Indeed.com











    var tellFriendEmail = '';


    @import url(/styles/component/resumematch.css);.btn,.sg-btn{display:inline-block;padding:9px 15px;border:1px solid #9a99ac;border-bottom-color:#a2a2a2;-webkit-border-radius:6px;-moz-border-radius:6px;-ms-border-radius:6px;-o-border-radius:6px;border-radius:6px;background-color:#D9D9E2;background-image:-moz-linear-gradient(top, #f8f8f9, #D9D9E2);background-image:-webkit-gradient(linear, 0 0, 0 100%, from(#f8f8f9), to(#D9D9E2));background-image:-webkit-linear-gradient(top, #f8f8f9, #D9D9E2);background-image:linear-gradient(to bottom, #f8f8f9, #D9D9E2);background-repeat:repeat-x;-webkit-box-shadow:0 1px 5px rgba(0,0,0,0.2);-moz-box-shadow:0 1px 5px rgba(0,0,0,0.2);-ms-box-shadow:0 1px 5px rgba(0,0,0,0.2);-o-box-shadow:0 1px 5px rgba(0,0,0,0.2);box-shadow:0 1px 5px rgba(0,0,0,0.2);color:#333;vertical-align:middle;text-align:center;text-decoration:none;text-shadow:0 1px #fff;font-wei

In [29]:
soup

<!DOCTYPE html>

<html dir="ltr" lang="en">
<head><title>Analytic Data Scientist job - IBM - Denver, CO | Indeed.com</title>
<meta content="May 31, 2018 - Analytic Data Scientist - div  b Job Description /b  br  IBM Global Business Services (GBS) is a team of business, strategy and technology consultants enabling enterprises to" name="description"/>
<meta content="Analytic Data Scientist job - IBM - Denver, CO" name="keywords"/>
<meta content="NOINDEX" name="robots"/>
<link href="/m/viewjob?jk=da4412e0f36920a0" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="/m/viewjob?jk=da4412e0f36920a0" media="handheld" rel="alternate"/>
<meta content="origin-when-cross-origin" name="referrer"/>
<meta content="115882278440564" property="fb:app_id"/>
<meta content="Analytic Data Scientist job - IBM - Denver, CO" property="og:title"/>
<meta content="Indeed.com" property="og:site_name"/>
<meta content="http://www.indeed.com/images/medium_logo.png" property="og:image"/>
<script 